In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
import pandas as pd

In [4]:
driver = webdriver.Chrome()
driver.implicitly_wait(7)  # wait up until 7 seconds for the next requested element to appear before erroring out.

In [98]:

goto_main_calendar = lambda: driver.get("https://laikra.komm.one/dvvlaikraIGV21/servlet/Internetgeschaeftsvorfaelle?AUFRUF=TMR_lhs")
get_schalter13_button = lambda: driver.find_element(by='name', value='BTN_SELECT_0001_TERMINORT_AUSWAHL')
get_schalter14_button = lambda: driver.find_element(by='name', value='BTN_SELECT_0002_TERMINORT_AUSWAHL')
get_schalter_title = lambda schalter: schalter.get_attribute('title')
# check_calandar_opendays = lambda: driver.find_element(by='id', value='CAL_TERMINSUCHE_DATUM_SUCHEN').find_element(by=By.CLASS_NAME, value='offen')

get_calandar_opendays = lambda: driver.find_element(by='id', value='CAL_TERMINSUCHE_DATUM_SUCHEN').find_elements(by=By.CLASS_NAME, value='offen') + driver.find_element(by='id', value='CAL_TERMINSUCHE_DATUM_SUCHEN').find_elements(by=By.CLASS_NAME, value='aktuell')
get_date_from_day = lambda day: day.get_attribute('title')
get_timeslots = lambda: driver.find_element(by=By.ID, value='CAL_TERMNSUCHE_ZEIT_AUSWAEHLEN').find_elements(by=By.TAG_NAME, value='input')
get_info_from_timeslot = lambda timeslot: (timeslot.get_attribute('title'), timeslot.get_attribute('class'))
get_next_month_button = lambda: driver.find_element(by=By.NAME, value='BTN_KALENDERPLUS_TERMINDATUM_AUSWAHL')


In [102]:
dates = []
print('starting')


for get_schalter_button in [get_schalter13_button, get_schalter14_button]:  
    goto_main_calendar()
    while True:
        schalter_button = get_schalter_button()
        schalter = get_schalter_title(schalter_button)
        print(schalter)
        schalter_button.click()
        for idx, _ in enumerate(get_calandar_opendays()):
            day_selector = get_calandar_opendays()[idx]  # repeat to make sure it's a valid link
            date = get_date_from_day(day_selector)
            print(date)
            day_selector.click()
            for timeslot in get_timeslots():
                info = get_info_from_timeslot(timeslot)
                print(info)
                full_info = {
                    'schalter': schalter, 
                    'date': date, 
                    'time': info[0], 
                    'status': info[1]
                }
                print(full_info)
                dates.append(full_info)
                
        # if there is another month, then click it and repeat the process.  otherwise, just skip to the next schalter.
        next_month_button = get_next_month_button()
        if 'disabled' in next_month_button.get_attribute('class'):
            break
        else:
            next_month_button.click()
            


starting
Schalter 13
25.04.2022
('13:30', 'geschlossen')
{'schalter': 'Schalter 13', 'date': '25.04.2022', 'time': '13:30', 'status': 'geschlossen'}
('13:45', 'geschlossen')
{'schalter': 'Schalter 13', 'date': '25.04.2022', 'time': '13:45', 'status': 'geschlossen'}
('14:00', 'geschlossen')
{'schalter': 'Schalter 13', 'date': '25.04.2022', 'time': '14:00', 'status': 'geschlossen'}
('14:15', 'geschlossen')
{'schalter': 'Schalter 13', 'date': '25.04.2022', 'time': '14:15', 'status': 'geschlossen'}
('14:30', 'geschlossen')
{'schalter': 'Schalter 13', 'date': '25.04.2022', 'time': '14:30', 'status': 'geschlossen'}
('14:45', 'geschlossen')
{'schalter': 'Schalter 13', 'date': '25.04.2022', 'time': '14:45', 'status': 'geschlossen'}
29.04.2022
('10:15', 'geschlossen')
{'schalter': 'Schalter 13', 'date': '29.04.2022', 'time': '10:15', 'status': 'geschlossen'}
('10:30', 'geschlossen')
{'schalter': 'Schalter 13', 'date': '29.04.2022', 'time': '10:30', 'status': 'geschlossen'}
('10:45', 'geschlosse

In [105]:
df = pd.DataFrame(dates)
df

,schalter,date,time,status
0,Schalter 13,25.04.2022,13:30,geschlossen
1,Schalter 13,25.04.2022,13:45,geschlossen
2,Schalter 13,25.04.2022,14:00,geschlossen
3,Schalter 13,25.04.2022,14:15,geschlossen
4,Schalter 13,25.04.2022,14:30,geschlossen
...,...,...,...,...
377,Schalter 14,23.05.2022,09:30,offen
378,Schalter 14,23.05.2022,09:45,offen
379,Schalter 14,23.05.2022,10:00,geschlossen
380,Schalter 14,23.05.2022,vor 07:15 Uhr,disabledErster


In [107]:
df[df.status == 'offen']

,schalter,date,time,status
165,Schalter 13,23.05.2022,07:45,offen
170,Schalter 13,23.05.2022,09:00,offen
171,Schalter 13,23.05.2022,09:15,offen
172,Schalter 13,23.05.2022,09:30,offen
179,Schalter 13,23.05.2022,07:45,offen
184,Schalter 13,23.05.2022,09:00,offen
185,Schalter 13,23.05.2022,09:15,offen
186,Schalter 13,23.05.2022,09:30,offen
362,Schalter 14,23.05.2022,09:15,offen
363,Schalter 14,23.05.2022,09:30,offen


In [103]:
dates

[{'schalter': 'Schalter 13',
  'date': '25.04.2022',
  'time': '13:30',
  'status': 'geschlossen'},
 {'schalter': 'Schalter 13',
  'date': '25.04.2022',
  'time': '13:45',
  'status': 'geschlossen'},
 {'schalter': 'Schalter 13',
  'date': '25.04.2022',
  'time': '14:00',
  'status': 'geschlossen'},
 {'schalter': 'Schalter 13',
  'date': '25.04.2022',
  'time': '14:15',
  'status': 'geschlossen'},
 {'schalter': 'Schalter 13',
  'date': '25.04.2022',
  'time': '14:30',
  'status': 'geschlossen'},
 {'schalter': 'Schalter 13',
  'date': '25.04.2022',
  'time': '14:45',
  'status': 'geschlossen'},
 {'schalter': 'Schalter 13',
  'date': '29.04.2022',
  'time': '10:15',
  'status': 'geschlossen'},
 {'schalter': 'Schalter 13',
  'date': '29.04.2022',
  'time': '10:30',
  'status': 'geschlossen'},
 {'schalter': 'Schalter 13',
  'date': '29.04.2022',
  'time': '10:45',
  'status': 'geschlossen'},
 {'schalter': 'Schalter 13',
  'date': '29.04.2022',
  'time': '11:00',
  'status': 'geschlossen'},
